# Use Intent Classification to orchestrate multiple sub-agents 

#### Sample dialog 

1. what's my balance [asks for customer id]
2. cool, when's my next pmt due 
3. refi rates are down to 3.5. dude, should i refi?
4. I'm also applying for a new mortgage. can you remind me if I've got any other docs I'm supposed to get you?

First step is to install the pre-requisites packages

In [59]:
# !pip install --upgrade -q -r requirements.txt
# !pip install --upgrade -q boto3 botocore 

In [1]:
import time
import boto3
import logging
import pprint
import json

%load_ext autoreload
%autoreload 2

from agent import AgentsForAmazonBedrock

In [2]:
#Clients
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
bedrock_client = boto3.client('bedrock-runtime')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]

suffix = f"{region}-{account_id}"
bucket_name = f'mac-workshop-{suffix}'
agent_foundation_models = [
    "anthropic.claude-3-haiku-20240307-v1:0",
    "anthropic.claude-3-sonnet-20240229-v1:0", 
    ]

In [4]:
agents = AgentsForAmazonBedrock()

In [5]:
sub_agent_names = ["existing_mortgage_agent", "mortgage_application_agent"]

In [6]:
def classifyIntentByLLM(input_text, bedrock_runtime, model_id, 
                        top_p=1, temp=0, system=''):  
    messages = [
        {
        "role": 'user',
        "content": [ {"type": "text", "text":
f"""
You will be given some user input. Classify that input according to which bot would be
the most appropriate one to respond that input.

EXISTING: can retrieve the latest information about an existing mortgage, like principal balance and
payment information.

APPLICATION: handle requests related to mortgage applications that are in process. creating, managing, 
and completing these mortgage applications. help find status of required application documentation.
assumes that the mortgage type is already determined. unable to handle general questions about mortgages
like tradeoffs between different mortgage types.

OTHER: not applicable for any of the bots available.

Here is the user input text:
<input_text>
{input_text}
</input_text>

Return only the name of the bot, with no preamble.
"""
        }]
        }
    ]  
    body=json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 25,
            "messages": messages,
            "temperature": temp,
            "top_p": top_p,
            "system": system,
            "stop_sequences":["assistant"]
        }  
    )  
    
    response = bedrock_runtime.invoke_model(body=body, modelId=model_id)
    response_body = json.loads(response.get('body').read())
    # print(response_body)
    
    return response_body['content'][0]['text']

In [7]:
classifyIntentByLLM("what is the status of my application?", bedrock_client, 
                model_id = "anthropic.claude-3-haiku-20240307-v1:0", temp=0.5, top_p=0.9)

'APPLICATION'

In [8]:
classifyIntentByLLM("what is better, 30 year mortgage or 15 year?", bedrock_client, 
                model_id = "anthropic.claude-3-haiku-20240307-v1:0", temp=0.5, top_p=0.9)

'OTHER'

In [9]:
classifyIntentByLLM("why would I pick a 15 year mortgage over a 30 year?", bedrock_client, 
                model_id = "anthropic.claude-3-haiku-20240307-v1:0", temp=0.5, top_p=0.9)

'OTHER'

In [10]:
classifyIntentByLLM("what is the status of the documents I need to provide?", bedrock_client, 
                model_id = "anthropic.claude-3-haiku-20240307-v1:0", temp=0.5, top_p=0.9)

'APPLICATION'

In [11]:
classifyIntentByLLM("when is my next payment due?", bedrock_client, 
                model_id = "anthropic.claude-3-haiku-20240307-v1:0", temp=0.5, top_p=0.9)

'EXISTING'

In [12]:
classifyIntentByLLM("I am customer 288. What is my existing balance?", bedrock_client, 
                model_id = "anthropic.claude-3-haiku-20240307-v1:0", temp=0.5, top_p=0.9)

'EXISTING'

In [13]:
INTENT_TO_ID_MAP = {
    "EXISTING": agents.get_agent_id_by_name("existing_mortgage_agent"),
    "APPLICATION": agents.get_agent_id_by_name("mortgage_application_agent")
}
INTENT_TO_ID_MAP 

{'EXISTING': 'VHJGRWYVLS', 'APPLICATION': 'N5Y3BFKPTD'}

In [14]:
def invokeByIntent(input_text, session_id, session_state={}):
    input_intent = classifyIntentByLLM(input_text, bedrock_client, 
                            model_id = "anthropic.claude-3-haiku-20240307-v1:0", temp=0.5, top_p=0.9)

    if input_intent == "EXISTING":
        # print('invoking Existing Mortgage agent...')
        response = agents.invoke(input_text, INTENT_TO_ID_MAP["EXISTING"], 
                                              session_id=session_id, session_state=session_state)
    elif input_intent == "APPLICATION":
        # print('invoking Mortgage Application agent...')
        response = agents.invoke(input_text, INTENT_TO_ID_MAP["APPLICATION"], 
                                              session_id=session_id, session_state=session_state)
    else:
        response = "Sorry, I don't have that expertise. Please ask a different question."

    return response

In [15]:
sup_session_id = '999'

In [16]:
%%time
print(invokeByIntent("my id is 8953. when is my payment due?", sup_session_id))

According to the mortgage information, your next payment is due on 2024-07-01.
CPU times: user 17.1 ms, sys: 2.41 ms, total: 19.5 ms
Wall time: 2.85 s


In [17]:
%%time
invokeByIntent("why would I pick a 15 year mortgage over a 30?", sup_session_id)

CPU times: user 2.02 ms, sys: 1.07 ms, total: 3.09 ms
Wall time: 346 ms


"Sorry, I don't have that expertise. Please ask a different question."

In [18]:
%%time
invokeByIntent("I am customer 948. what documents do i still need to provide for my application?", 
                sup_session_id)

CPU times: user 4.64 ms, sys: 1.1 ms, total: 5.74 ms
Wall time: 3.83 s


'According to the mortgage application document status for customer 948, the only document you still need to provide is your employment information. All other required documents, including proof of income, proof of assets, and credit information, have been completed.'

In [19]:
%%time
invokeByIntent("what documents have i already provided for my application?", sup_session_id)

CPU times: user 3.38 ms, sys: 964 µs, total: 4.35 ms
Wall time: 2.9 s


'According to the mortgage application document status, you have already provided the following documents for your application:\n- Proof of income\n- Proof of assets \n- Credit information\n\nThe only document you have not yet provided is your employment information.'

In [20]:
%%time
invokeByIntent("my ID is 9856. what is my principal balance on my existing mortgage?", 
            sup_session_id)

CPU times: user 3.82 ms, sys: 1.03 ms, total: 4.85 ms
Wall time: 2.47 s


'The outstanding principal balance on your existing mortgage is $150,000.00.'

In [22]:
%%time 
invokeByIntent("my customer id is 8953. when is my payment due?", sup_session_id)

invokeByIntent
my customer id is 8953. when is my payment due?
invoking Existing Mortgage agent...
CPU times: user 5.42 ms, sys: 1.44 ms, total: 6.86 ms
Wall time: 2.07 s


'According to the mortgage status information for customer ID 8953, your next mortgage payment is due on 2024-07-01.'

In [21]:
%%time 
invokeByIntent("what are the cons of a 30 year mortgage?", sup_session_id)

CPU times: user 2.28 ms, sys: 1.2 ms, total: 3.48 ms
Wall time: 270 ms


"Sorry, I don't have that expertise. Please ask a different question."

##### Invoke Agent by Intent, with prompt attribute

In [22]:
from datetime import datetime
today = datetime.today().strftime('%b-%d-%Y')

session_state = {
    "promptSessionAttributes": {
        "customer_ID": "498",
        "today": today
    }
}
session_state

{'promptSessionAttributes': {'customer_ID': '498', 'today': 'Jun-01-2024'}}

In [23]:
%%time
sup_session_id = "111"
print(invokeByIntent("what docs do I still owe you?", 
                    sup_session_id, session_state=session_state))

According to the information I have, you have already provided the following documents for your mortgage application:
- Proof of income
- Proof of assets

However, you still need to provide the following document:
- Employment information

Please submit the missing employment information document as soon as possible so we can continue processing your mortgage application.
CPU times: user 5.02 ms, sys: 1.52 ms, total: 6.54 ms
Wall time: 3.31 s


In [24]:
%%time
sup_session_id = "222"
print(invokeByIntent("how many years until I reach the mortgage maturity date?", 
                        sup_session_id, session_state=session_state))

The number of years until the mortgage maturity date is 6 years.
CPU times: user 4.82 ms, sys: 1.36 ms, total: 6.19 ms
Wall time: 3.45 s


### Quick performance test

In [25]:
import uuid 
import time
import numpy as np

def query_loop_by_intent(query, num_invokes):
    latencies = []
    for i in range(num_invokes):
        _session_id = str(uuid.uuid1())
        _start_time = time.time()
        resp = invokeByIntent(query, _session_id)
        _end_time = time.time()
        latencies.append(_end_time - _start_time)

    print(f'\n\nInvoked by intent {num_invokes} times.')
    # get sum of total latencies
    total_time = sum(latencies)
    # get average latency
    avg_time = total_time / num_invokes
    # get p90 latency
    p90_time = np.percentile(latencies, 90)

    print(f'Average latency: {avg_time:.1f}, P90 latency: {p90_time:.1f}')

In [26]:
query_loop_by_intent("I am customer 999. how many years until the mortgage maturity date?", 25)



Invoked by intent 25 times.
Average latency: 3.1, P90 latency: 3.3


In [27]:
import uuid 
import time
import numpy as np

def classification_loop(query, num_invokes, runtime, model):
    latencies = []
    for i in range(num_invokes):
        _session_id = str(uuid.uuid1())
        _start_time = time.time()
        resp = classifyIntentByLLM(query, runtime, model)
        _end_time = time.time()
        latencies.append(_end_time - _start_time)

    print(f'\n\nClassified by intent {num_invokes} times.')
    # get sum of total latencies
    total_time = sum(latencies)
    # get average latency
    avg_time = total_time / num_invokes
    # get p90 latency
    p90_time = np.percentile(latencies, 90)

    print(f'Average latency: {avg_time:.1f}, P90 latency: {p90_time:.1f}')

In [28]:
classification_loop("I'm customer 99. what's my balance?", 25, 
                    bedrock_client, agent_foundation_models[0])



Classified by intent 25 times.
Average latency: 0.3, P90 latency: 0.4
